In [ ]:
# === EDIT THIS: your MIMIC-IV folder ===
MIMIC_DIR = "/kaggle/input/pfe-data"  # e.g., "/content/drive/MyDrive/mimic-iv-2.2"

# Core tables
ADMISSIONS_CSV = '/kaggle/input/pfe-data/admissions.csv'
PATIENTS_CSV   = '/kaggle/input/pfe-data/patients.csv'
ICUSTAYS_CSV   = '/kaggle/input/pfe-data/icustays.csv'

# Events + dictionaries
CHARTEVENTS_CSV = '/kaggle/input/pfe-data/chartevents.csv'
D_ITEMS_CSV     = '/kaggle/input/ditems/d_items.csv'

LABEVENTS_CSV   = '/kaggle/input/pfe-data/labevents.csv'
D_LABITEMS_CSV  = '/kaggle/input/hhhhhh/d_labitems.csv'

import pandas as pd, numpy as np, gc, math, os, re
from collections import defaultdict
pd.set_option("display.max_colwidth", 120)

SEED = 42
np.random.seed(SEED)


In [ ]:
# Read minimal columns (faster)
adm_cols = ["subject_id","hadm_id","admittime","dischtime","deathtime","hospital_expire_flag"]
icu_cols = ["subject_id","hadm_id","stay_id","first_careunit","intime","outtime","los"]
pat_cols = ["subject_id","anchor_age","anchor_year"]

adm = pd.read_csv(ADMISSIONS_CSV, usecols=adm_cols, parse_dates=["admittime","dischtime","deathtime"],
                  dtype={"subject_id":"int32","hadm_id":"int32","hospital_expire_flag":"int8"})
icu = pd.read_csv(ICUSTAYS_CSV,  usecols=icu_cols, parse_dates=["intime","outtime"],
                  dtype={"subject_id":"int32","hadm_id":"int32","stay_id":"int32","first_careunit":"category"})
pat = pd.read_csv(PATIENTS_CSV,  usecols=pat_cols, dtype={"subject_id":"int32","anchor_year":"int16","anchor_age":"float32"})

# Age at admission (standard for MIMIC-IV)
adm_year = adm["admittime"].dt.year.astype("Int16")
tmp = adm.merge(pat, on="subject_id", how="left")
tmp["age_at_admit"] = tmp["anchor_age"] + (adm_year.astype("float32") - tmp["anchor_year"].astype("float32"))
tmp["age_at_admit"] = tmp["age_at_admit"].clip(lower=0, upper=120)

# Merge ICU with admission info
cohort = icu.merge(
    tmp[["subject_id","hadm_id","admittime","dischtime","deathtime","hospital_expire_flag","age_at_admit"]],
    on=["subject_id","hadm_id"], how="left"
)

# Inclusion criteria
cohort["icu_los_hours"] = (cohort["outtime"] - cohort["intime"]).dt.total_seconds() / 3600.0
cohort = cohort[cohort["age_at_admit"] >= 18].copy()
cohort = cohort[cohort["icu_los_hours"] >= 24].copy()
cohort = cohort.sort_values(["subject_id","hadm_id","intime"]).drop_duplicates(["subject_id","hadm_id"], keep="first")

# Label: in-hospital death AFTER 24h from ICU intime
delta_death = (cohort["deathtime"] - cohort["intime"]).dt.total_seconds() / 3600.0
early_death_mask = (cohort["hospital_expire_flag"] == 1) & (cohort["deathtime"].notna()) & (delta_death <= 24)
cohort = cohort[~early_death_mask].copy()  # drop deaths in first 24h

delta_death = (cohort["deathtime"] - cohort["intime"]).dt.total_seconds() / 3600.0
cohort["label"] = ((cohort["hospital_expire_flag"] == 1) & (cohort["deathtime"].notna()) & (delta_death > 24)).astype(int)

# Metadata for FL
cohort["careunit"] = cohort["first_careunit"].astype(str)
cohort["adm_year"] = cohort["admittime"].dt.year.astype("Int16")

print("Cohort size:", len(cohort))
print("Pos rate:", cohort["label"].mean().round(4))
print(cohort[["careunit"]].value_counts().head())


In [ ]:
cohort = cohort.reset_index(drop=True)
stay_to_start = pd.Series(cohort["intime"].values, index=cohort["stay_id"].values)
stay_to_end   = stay_to_start + pd.Timedelta(hours=24)

stay_ids = set(cohort["stay_id"].tolist())
hadm_to_intime = cohort.set_index("hadm_id")["intime"]  # for labevents windowing


In [ ]:
# --- Vital signs from icu/d_items ---
d_items = pd.read_csv(D_ITEMS_CSV, usecols=["itemid","label","category"], dtype={"itemid":"int32","label":"string","category":"string"})
d_items["label_lc"] = d_items["label"].str.lower()

def find_itemids(patterns, in_category=None):
    m = pd.Series(False, index=d_items.index)
    for pat in patterns:
        m |= d_items["label_lc"].str.contains(pat, regex=True, na=False)
    if in_category is not None:
        m &= d_items["category"].str.contains(in_category, case=False, na=False)
    return set(d_items.loc[m, "itemid"].astype(int).tolist())

vital_map = {
    # var_name : regex patterns matched on d_items.label
    "hr":   find_itemids([r"\bheart rate\b"], in_category="vital"),            # Heart Rate
    "rr":   find_itemids([r"\brespiratory rate\b"], in_category="vital"),      # Respiratory Rate
    "spo2": find_itemids([r"\bspo2\b", r"oxygen saturation", r"pulse ox"], in_category="vital"),
    "temp": find_itemids([r"\btemperature\b"], in_category="vital"),
    "map":  find_itemids([r"\bmean arterial pressure\b", r"\bmap\b", r"non invasive blood pressure mean", r"arterial blood pressure mean"]),
    "sbp":  find_itemids([r"\bsystolic\b"], in_category="vital"),
    "dbp":  find_itemids([r"\bdiastolic\b"], in_category="vital"),
    "gcs":  find_itemids([r"\bglasgow coma scale total\b", r"\bgcs total\b"]),
}
# Remove empties
vital_map = {k:v for k,v in vital_map.items() if len(v)>0}
sum_ids_vital = set().union(*vital_map.values())
print({k: len(v) for k,v in vital_map.items()})

# --- Lab tests from hosp/d_labitems ---
d_lab = pd.read_csv(D_LABITEMS_CSV, usecols=["itemid","label"], dtype={"itemid":"int32","label":"string"})
d_lab["label_lc"] = d_lab["label"].str.lower()

def find_lab_itemids(patterns):
    m = pd.Series(False, index=d_lab.index)
    for pat in patterns:
        m |= d_lab["label_lc"].str.contains(pat, regex=True, na=False)
    return set(d_lab.loc[m, "itemid"].astype(int).tolist())

lab_map = {
    "wbc":        find_lab_itemids([r"\bwbc\b", r"white blood cell"]),
    "hgb":        find_lab_itemids([r"\bhemoglobin\b"]),
    "platelets":  find_lab_itemids([r"\bplatelet"]),
    "lactate":    find_lab_itemids([r"\blactate\b"]),
    "sodium":     find_lab_itemids([r"\bsodium\b"]),
    "potassium":  find_lab_itemids([r"\bpotassium\b"]),
    "chloride":   find_lab_itemids([r"\bchloride\b"]),
    "bicarb":     find_lab_itemids([r"\bbicarbonate\b", r"\bHCO3\b"]),
    "creatinine": find_lab_itemids([r"\bcreatinine\b"]),
    "bun":        find_lab_itemids([r"\burea\b", r"\bbun\b"]),
    "glucose":    find_lab_itemids([r"\bglucose\b"]),
    "bilirubin":  find_lab_itemids([r"\bbilirubin\b"]),
    "inr":        find_lab_itemids([r"\binr\b"]),
}
lab_map = {k:v for k,v in lab_map.items() if len(v)>0}
sum_ids_lab = set().union(*lab_map.values())
print({k: len(v) for k,v in lab_map.items()})


In [ ]:
# Online aggregator per (stay_id, var)
class OnlineAgg:
    __slots__ = ("n","sum","sumsq","min","max","last_t","last_v")
    def __init__(self):
        self.n=0; self.sum=0.0; self.sumsq=0.0
        self.min=float("inf"); self.max=float("-inf")
        self.last_t=pd.Timestamp.min; self.last_v=np.nan
    def update(self, t, v):
        if pd.isna(v): return
        self.n += 1
        self.sum += v
        self.sumsq += v*v
        if v < self.min: self.min = v
        if v > self.max: self.max = v
        if t > self.last_t: 
            self.last_t = t
            self.last_v = v
    def to_dict(self):
        mean = self.sum/self.n if self.n>0 else np.nan
        var = (self.sumsq/self.n - mean*mean) if self.n>1 else np.nan
        std = math.sqrt(var) if var==var and var>=0 else np.nan
        return {"count": self.n, "mean": mean, "std": std, "min": (self.min if self.n>0 else np.nan),
                "max": (self.max if self.n>0 else np.nan), "last": self.last_v}

# Build itemid -> varname map for fast lookup
item_to_var = {}
for var, ids in vital_map.items():
    for iid in ids: item_to_var[iid] = var

agg_vitals = defaultdict(OnlineAgg)

usecols_ce = ["stay_id","itemid","charttime","valuenum"]
dtype_ce = {"stay_id":"int32","itemid":"int32","valuenum":"float32"}

chunksize = 10_000_000  # tune for your RAM
keep_ids = set(stay_ids)

print("Streaming chartevents ...")
for chunk in pd.read_csv(CHARTEVENTS_CSV, usecols=usecols_ce, dtype=dtype_ce,
                         parse_dates=["charttime"], chunksize=chunksize):
    # Filter to our stays & itemids of interest
    chunk = chunk[chunk["stay_id"].isin(keep_ids)]
    chunk = chunk[chunk["itemid"].isin(sum_ids_vital)]
    if chunk.empty:
        continue

    # Map 24h window per stay
    chunk["start"] = chunk["stay_id"].map(stay_to_start)
    chunk["end"]   = chunk["stay_id"].map(stay_to_end)
    m = (chunk["charttime"] >= chunk["start"]) & (chunk["charttime"] <= chunk["end"])
    chunk = chunk[m]
    if chunk.empty:
        continue

    # Update online aggregates
    for iid, sid, t, v in zip(chunk["itemid"].values, chunk["stay_id"].values, chunk["charttime"].values, chunk["valuenum"].values):
        var = item_to_var.get(int(iid))
        if var is None: 
            continue
        agg_vitals[(int(sid), var)].update(pd.Timestamp(t), float(v))

    del chunk
    gc.collect()

# Convert aggregates to a wide feature table
rows = []
for (sid, var), A in agg_vitals.items():
    d = A.to_dict()
    row = {"stay_id": sid}
    for k,v in d.items():
        row[f"{var}_{k}_0_24h"] = v
    rows.append(row)

fe_vitals = pd.DataFrame(rows).groupby("stay_id", as_index=False).first()  # one row per stay
print("Vitals features:", fe_vitals.shape)


In [ ]:
# --- ROBUST LABEVENTS STREAMING (handles NA hadm_id/itemid) ---
from collections import defaultdict
import math, gc

# lab itemid -> varname (already built above as `labitem_to_var`)
# hadm_to_intime (built earlier)
# cohort, lab_map, sum_ids_lab (sets) already defined

agg_labs = defaultdict(OnlineAgg)

usecols_le = ["hadm_id","itemid","charttime","valuenum"]
# Only enforce dtype on valuenum to avoid NA casting errors for IDs
dtype_le = {"valuenum":"float32"}

keep_hadms = set(cohort["hadm_id"].astype(int).tolist())
sum_ids_lab_int = set(int(i) for i in sum_ids_lab)  # ensure ints

chunksize = 1_000_000  # reduce if RAM is tight

print("Streaming labevents (robust) ...")
for chunk in pd.read_csv(
    LABEVENTS_CSV,
    usecols=usecols_le,
    dtype=dtype_le,                # don't force int dtypes here
    parse_dates=["charttime"],
    chunksize=chunksize,
    low_memory=True,
):
    # Coerce hadm_id/itemid to numeric with NA allowed
    chunk["hadm_id"] = pd.to_numeric(chunk["hadm_id"], errors="coerce")
    chunk["itemid"]  = pd.to_numeric(chunk["itemid"], errors="coerce")

    # Drop rows missing the essentials
    chunk = chunk.dropna(subset=["hadm_id","itemid","charttime"])
    if chunk.empty:
        continue

    # Now safe to cast to ints
    chunk["hadm_id"] = chunk["hadm_id"].astype("int32")
    chunk["itemid"]  = chunk["itemid"].astype("int32")

    # Filter by our cohort's hadm_ids and selected lab itemids
    chunk = chunk[chunk["hadm_id"].isin(keep_hadms)]
    if chunk.empty:
        continue
    chunk = chunk[chunk["itemid"].isin(sum_ids_lab_int)]
    if chunk.empty:
        continue

    # Map ICU intime for the hadm (first ICU stay we kept)
    chunk["icuintime"] = chunk["hadm_id"].map(hadm_to_intime)

    # Keep labs within 0–24h of ICU intime
    end = chunk["icuintime"] + pd.Timedelta(hours=24)
    m = (chunk["charttime"] >= chunk["icuintime"]) & (chunk["charttime"] <= end)
    chunk = chunk[m]
    if chunk.empty:
        continue

    # Map to stay_id via hadm_id (1:1 after we kept first ICU stay per hadm)
    if "hadm_to_stay" not in globals():
        hadm_to_stay = cohort.set_index("hadm_id")["stay_id"].to_dict()
    chunk["stay_id"] = chunk["hadm_id"].map(hadm_to_stay)

    # Drop any rows that still lack a stay_id (should be rare)
    chunk = chunk.dropna(subset=["stay_id"])
    if chunk.empty:
        continue

    # Aggregate online
    for iid, sid, t, v in zip(
        chunk["itemid"].values,
        chunk["stay_id"].astype("int32").values,
        chunk["charttime"].values,
        chunk["valuenum"].values,
    ):
        # Skip non-numeric lab values (valuenum NA)
        if pd.isna(v):
            continue
        var = labitem_to_var.get(int(iid))
        if var is None:
            continue
        agg_labs[(int(sid), var)].update(pd.Timestamp(t), float(v))

    del chunk
    gc.collect()

# Build wide lab feature table
rows = []
for (sid, var), A in agg_labs.items():
    d = A.to_dict()
    row = {"stay_id": sid}
    for k,v in d.items():
        row[f"lab_{var}_{k}_0_24h"] = v
    rows.append(row)

fe_labs = pd.DataFrame(rows).groupby("stay_id", as_index=False).first()
print("Lab features:", fe_labs.shape)


In [ ]:
# Minimal cohort columns for modeling/partitioning
base_cols = [
    "subject_id","hadm_id","stay_id",
    "careunit","adm_year",
    "intime","outtime","admittime","dischtime","deathtime",
    "icu_los_hours","age_at_admit",
    "label",
]
base = cohort[base_cols].copy()

# Merge features
final = base.merge(fe_vitals, on="stay_id", how="left").merge(fe_labs, on="stay_id", how="left")

# Optional missingness flags (helps models + later imputation)
for c in final.columns:
    if c.endswith("_0_24h") and ("count" not in c):
        final[c.replace("_0_24h","_missing_0_24h")] = final[c].isna().astype("int8")

print("Final shape:", final.shape)
print("Label prevalence:", final["label"].mean().round(4))


In [ ]:
OUT_PATH = "/kaggle/working/mimic_iv_0_24h_features.parquet"
final.to_parquet(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
